# Data Analytics for Finance: Final Project

**Student Name:** Omar Ben Ayed

**Student Number:** 38628

**Signal Name (e.g., Book to Market):** Piotroski F-score

**Signal Code (e.g., be_me):** f_score

## Imports and initial adjustments

In [ ]:
#r "nuget: FSharp.Data"
#r "nuget: FSharp.Stats"
#r "nuget: Plotly.NET,2.0.0-preview.17"
#r "nuget: Plotly.NET.Interactive,2.0.0-preview.17"
#r "nuget: FsExcel"
#r "nuget: DiffSharp-lite"
#r "nuget: Accord"
#r "nuget: Accord.Statistics"
#load "Common.fsx"
#load "YahooFinance.fsx"
#load "Portfolio.fsx"
open Portfolio
open Common
open YahooFinance
open Accord
open Accord.Statistics.Models.Regression.Linear
open System.IO
open FsExcel
open System
open FSharp.Data
open FSharp.Stats
open Plotly.NET
open DiffSharp

Installed Packages Accord, 3.8.0 Accord.Statistics, 3.8.0 DiffSharp-lite, 1.0.7 FsExcel, 0.0.37 FSharp.Data, 4.2.8 FSharp.Stats, 0.4.5 Plotly.NET, 2.0.0-preview.17 Plotly.NET.Interactive, 2.0.0-preview.17

Loading extensions from `Plotly.NET.Interactive.dll`

In [ ]:
// Set dotnet interactive formatter to plaintext
Formatter.Register(fun (x:obj) (writer: TextWriter) -> fprintfn writer "%120A" x )
Formatter.SetPreferredMimeTypesFor(typeof<obj>, "text/plain")
// Make plotly graphs work with interactive plaintext formatter
Formatter.SetPreferredMimeTypesFor(typeof<GenericChart.GenericChart>,"text/html")

In [ ]:
// setting to current directory
let [<Literal>] ResolutionFolder = __SOURCE_DIRECTORY__
Environment.CurrentDirectory <- ResolutionFolder

In [ ]:
let [<Literal>] IdAndReturnsFilePath = "data/id_and_return_data.csv"
let [<Literal>] MySignalFilePath = "data/f_score.csv"

In [ ]:
type IdAndReturnsType = 
    CsvProvider<Sample=IdAndReturnsFilePath,
                // The schema parameter is not required,
                // but I am using it to override some column types
                // to make filtering easier.
                // If I didn't do this these particular columns 
                // would have strings of "1" or "0", but explicit boolean is nicer.
                Schema="obsMain(string)->obsMain=bool,exchMain(string)->exchMain=bool",
                ResolutionFolder=ResolutionFolder>

type MySignalType = 
    CsvProvider<MySignalFilePath,
                ResolutionFolder=ResolutionFolder>

In [ ]:
let idAndReturnsCsv = IdAndReturnsType.GetSample()

let mySignalCsv = MySignalType.GetSample()

## Overview

Available in the PDF file.

## Strategy analysis

### Strategy construction

In [ ]:
let idAndReturnsRows = idAndReturnsCsv.Rows |> Seq.toList
let mySignalRows = mySignalCsv.Rows |> Seq.toList

In [ ]:
type NonMissingSignal =
    {
        Id: string
        Eom: DateTime
        Signal: float
    }

let myNonMissingSignals =
    mySignalRows
    |> List.choose (fun row -> 
        match row.Signal with
        | None -> None
        | Some signal -> 
            Some { Id = row.Id; Eom = row.Eom; Signal = signal })

In [ ]:
let strategyName = "Piotroski F-Score"

let msfBySecurityIdAndMonth =
    idAndReturnsRows
    |> List.map(fun row -> 
        let id = Other row.Id
        let month = DateTime(row.Eom.Year,row.Eom.Month,1)
        let key = id, month
        key, row)
    |> Map    

let signalBySecurityIdAndMonth =
    mySignalRows
    |> List.choose(fun row -> 
        // we'll use choose to drop the security if the signal is None.
        // The signal is None when it is missing.
        match row.Signal with
        | None -> None // choose will drop these None observations
        | Some signal ->
            let id = Other row.Id
            let month = DateTime(row.Eom.Year,row.Eom.Month,1)
            let key = id, month
            // choose will convert Some(key,signal) into
            // (key,signal) and keep that.
            Some (key, signal))
    |> Map    

// securitiesByFormationMonth refers 
let securitiesByFormationMonth =
    idAndReturnsRows
    |> List.groupBy(fun x -> DateTime(x.Eom.Year, x.Eom.Month,1))
    |> List.map(fun (ym, obsThisMonth) -> 
        let idsThisMonth = [ for x in obsThisMonth do Other x.Id ]
        ym, idsThisMonth)
    |> Map

let getInvestmentUniverse formationMonth =
    match Map.tryFind formationMonth securitiesByFormationMonth with
    | Some securities -> 
        { FormationMonth = formationMonth 
          Securities = securities }
    | None -> failwith $"{formationMonth} is not in the date range"

In [ ]:
let securitiesByFormationMonth =
    idAndReturnsRows
    |> List.groupBy(fun x -> DateTime(x.Eom.Year, x.Eom.Month,1))
    |> List.map(fun (ym, obsThisMonth) -> 
        let idsThisMonth = [ for x in obsThisMonth do Other x.Id ]
        ym, idsThisMonth)
    |> Map

let getInvestmentUniverse formationMonth =
    match Map.tryFind formationMonth securitiesByFormationMonth with
    | Some securities -> 
        { FormationMonth = formationMonth 
          Securities = securities }
    | None -> failwith $"{formationMonth} is not in the date range"

In [ ]:
let getMySignal (securityId, formationMonth) =
    match Map.tryFind (securityId, formationMonth) signalBySecurityIdAndMonth with
    | None -> None
    | Some signal ->
        Some { SecurityId = securityId 
               // if a high signal means low returns,
               // use `-signal` here instead of `signal`
               Signal = signal }

In [ ]:
let getMySignals (investmentUniverse: InvestmentUniverse) =
    let listOfSecuritySignals =
        investmentUniverse.Securities
        |> List.choose(fun security -> 
            getMySignal (security, investmentUniverse.FormationMonth))    
    
    { FormationMonth = investmentUniverse.FormationMonth 
      Signals = listOfSecuritySignals }

let getMarketCap (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> None
    | Some row -> 
        match row.MarketEquity with
        | None -> None
        | Some me -> Some (security, me)

let getSecurityReturn (security, formationMonth) =
    // If the security has a missing return, assume that we got 0.0.
    // Note: If we were doing excess returns, we would need 0.0 - rf.
    let missingReturn = 0.0
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> security, missingReturn
    | Some x ->  
        match x.Ret with 
        | None -> security, missingReturn
        | Some r -> security, r

In [ ]:
let startSample = 
    idAndReturnsRows
    |> List.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
    |> List.min

let endSample = 
    let lastMonthWithData = 
        idAndReturnsRows
        |> Seq.map(fun row -> DateTime(row.Eom.Year,row.Eom.Month,1))
        |> Seq.max
    // The end of sample is the last month when we have returns.
    // So the last month when we can form portfolios is one month
    // before that.
    lastMonthWithData.AddMonths(-1) 

let sampleMonths = getSampleMonths (startSample, endSample)

In [ ]:
let isObsMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ObsMain

let isPrimarySecurity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.PrimarySec

let isCommonStock (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.Common

let isExchMain (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.ExchMain

let hasMarketEquity (security, formationMonth) =
    match Map.tryFind (security, formationMonth) msfBySecurityIdAndMonth with
    | None -> false
    | Some row -> row.MarketEquity.IsSome

let myFilters securityAndFormationMonth =
    isObsMain securityAndFormationMonth &&
    isPrimarySecurity securityAndFormationMonth &&
    isCommonStock securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    isExchMain securityAndFormationMonth &&
    hasMarketEquity securityAndFormationMonth

let doMyFilters (universe:InvestmentUniverse) =
    let filtered = 
        universe.Securities
        // my filters expect security, formationMonth
        |> List.map(fun security -> security, universe.FormationMonth)
        // do the filters
        |> List.filter myFilters
        // now convert back from security, formationMonth -> security
        |> List.map fst
    { universe with Securities = filtered }

In [ ]:
let formStrategy ym =
    ym
    |> getInvestmentUniverse
    |> doMyFilters
    |> getMySignals
    |> assignSignalSort strategyName 3
    |> List.map (giveValueWeights getMarketCap)
    |> List.map (getPortfolioReturn getSecurityReturn) 

let doParallel = true
let portfolios =
    if doParallel then
        sampleMonths
        |> List.toArray
        |> Array.Parallel.map formStrategy
        |> Array.toList
        |> List.collect id
    else
        sampleMonths
        |> List.collect formStrategy

### Performance analysis

In [ ]:
let ff3 = French.getFF3 Frequency.Monthly
let monthlyRiskFreeRate =
    [ for obs in ff3 do 
        let key = DateTime(obs.Date.Year,obs.Date.Month,1)
        key, obs.Rf ]
    |> Map

let portfolioExcessReturns =
    portfolios
    |> List.map(fun x -> 
        match Map.tryFind x.YearMonth monthlyRiskFreeRate with 
        | None -> failwith $"Can't find risk-free rate for {x.YearMonth}"
        | Some rf -> { x with Return = x.Return - rf })

In [ ]:
type StockData =
    { Symbol : string 
      Date : DateTime
      Return : float }

In [ ]:
let long = 
    portfolioExcessReturns 
    |> List.filter(fun x -> 
        x.PortfolioId = Indexed {| Name = strategyName; Index = 3 |})

In [ ]:
let vwMktRf =
    let portfolioMonths = 
        portfolioExcessReturns 
        |> List.map(fun x -> x.YearMonth)
    let minYm = portfolioMonths |> List.min
    let maxYm = portfolioMonths |> List.max
    
    [ for x in ff3 do
        if x.Date >= minYm && x.Date <= maxYm then
            { PortfolioId = Named("Mkt-Rf")
              YearMonth = x.Date
              Return = x.MktRf } ]

let short = 
    portfolioExcessReturns 
    |> List.filter(fun x -> 
        x.PortfolioId = Indexed {| Name = strategyName; Index = 1 |})

let longShort = 
    let shortByYearMonthMap = 
        short 
        |> List.map(fun row -> row.YearMonth, row) 
        |> Map
    
    [ for longObs in long do
        match Map.tryFind longObs.YearMonth shortByYearMonthMap with
        | None -> failwith "probably your date variables are not aligned for a weird reason"
        | Some shortObs ->
            { PortfolioId = Named "Long-Short"
              YearMonth = longObs.YearMonth
              Return = longObs.Return - shortObs.Return } ] 

In [ ]:
let cumulateSimpleReturn (xs: PortfolioReturn list) =
    let accumulator (priorObs:PortfolioReturn) (thisObs:PortfolioReturn) =
        let asOfNow = (1.0 + priorObs.Return)*(1.0 + thisObs.Return) - 1.0
        { thisObs with Return = asOfNow}
    // remember to make sure that your sort order is correct.
    match xs |> List.sortBy(fun x -> x.YearMonth) with
    | [] -> []      // return empty list if the input list is empty
    | head::tail -> // if there are observations do the calculation
        (head, tail) 
        ||> List.scan accumulator

In [ ]:
let portfolioReturnPlot (xs:PortfolioReturn list) =
    xs
    |> List.map(fun x -> x.YearMonth, x.Return)
    |> Chart.Line 
    |> Chart.withTitle "Growth of 1 Euro"

let chartStrategyAnalysis =
    List.concat [long; longShort; vwMktRf]
    |> List.groupBy(fun x -> x.PortfolioId)
    |> List.map(fun (portId, xs) ->
        xs
        |> cumulateSimpleReturn
        |> portfolioReturnPlot
        |> Chart.withTraceInfo (Name=portId.ToString())
        |> Chart.withTitle "Strategy Analysis: Market, Short and Long-Short")
    |> Chart.combine

In [ ]:
chartStrategyAnalysis

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let longStDev = 
    long |> Seq.stDevBy (fun x -> x.Return)

let longShortStDev =
    longShort |> Seq.stDevBy (fun x -> x.Return)

let vwMktRfStDev = 
    vwMktRf |> Seq.stDevBy (fun x -> x.Return)

let annualizeMonthlyStdDev monthlyStdDev: float  = sqrt(12.0) * monthlyStdDev

let longStDevA = 
    longStDev |> annualizeMonthlyStdDev

let longShortStDevA =
    longShortStDev |> annualizeMonthlyStdDev

let vwMktRfStDevA =
    vwMktRfStDev |> annualizeMonthlyStdDev

let long10 =
    let longWeight = (0.1 / longStDevA)
    [for x in long do
        { PortfolioId = x.PortfolioId
          YearMonth = x.YearMonth
          Return = x.Return * longWeight}]

let longShort10 =
    let longShortWeight = (0.1 / longShortStDevA)
    [for x in longShort do
        { PortfolioId = x.PortfolioId
          YearMonth = x.YearMonth
          Return = x.Return * longShortWeight}]

let vwMktRf10 =
    let vwMktRfWeight = (0.1 / vwMktRfStDevA)
    [for x in vwMktRf do
        { PortfolioId = x.PortfolioId
          YearMonth = x.YearMonth
          Return = x.Return * vwMktRfWeight}]

In [ ]:
let chartStrategyAnalysis10 =
    List.concat [long10; longShort10; vwMktRf10]
    |> List.groupBy(fun x -> x.PortfolioId)
    |> List.map(fun (portId, xs) ->
        xs
        |> cumulateSimpleReturn
        |> portfolioReturnPlot
        |> Chart.withTraceInfo (Name=portId.ToString())
        |> Chart.withTitle "Strategy Analysis: Market, Short and Long-Short (10% volatility)")
    |> Chart.combine

In [ ]:
chartStrategyAnalysis10

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let longListDivisor = 
    long |>
    List.map (fun x -> 
      {Symbol = "Long" 
       Date= x.YearMonth  
       Return= x.Return})

let longShortListDivisor = 
    longShort |>
    List.map (fun x -> 
      {Symbol = "Long Short" 
       Date= x.YearMonth  
       Return= x.Return})

let vwMktRfListDivisor = 
    vwMktRf |>
    List.map (fun x -> 
      {Symbol = "MRP" 
       Date= x.YearMonth  
       Return= x.Return})

let longCut =
    longListDivisor |> List.splitInto 2

let longShortCut =
    longShortListDivisor |> List.splitInto 2

let vwMktRfCut =
    vwMktRfListDivisor |> List.splitInto 2

let long1 = longCut[0]
let long2 = longCut[1]

let longShort1 = longShortCut[0]
let longShort2 = longShortCut[1]

let vwMktRf1 = vwMktRfCut[0]
let vwMktRf2 = vwMktRfCut[1]

let averageAnnualizedExcessReturn monthlyExcessReturn = 12.0 * monthlyExcessReturn

let annualizedSharpeRatio monthlySharpeRatio = sqrt(12.0) * monthlySharpeRatio

In [ ]:
let performanceLister list = 
    
    let listStDev = 
        list 
        |> Seq.stDevBy (fun x -> x.Return)
    
    let listReturn = 
        list 
        |> Seq.averageBy (fun x -> x.Return)
    
    let listSharpe = 
        listReturn / listStDev
    
    let listAnnualizedSharpeRatio = annualizedSharpeRatio listSharpe

    let listAnnualizedExcessReturn = averageAnnualizedExcessReturn listReturn

    // Tuple output 
    listAnnualizedExcessReturn, listAnnualizedSharpeRatio

In [ ]:
performanceLister long1, performanceLister long2, performanceLister longListDivisor

((0.008401573556, 0.05336235972), (0.1359395426, 0.9386818355), (0.07191649836, 0.4727976207))


In [ ]:
performanceLister longShort1, performanceLister longShort2, performanceLister longShortListDivisor

((0.03945217153, 0.4089443296), (-0.01665973611, -0.2717966971), (0.01150799442, 0.1418375176))


In [ ]:
performanceLister vwMktRf1, performanceLister vwMktRf2, performanceLister vwMktRfListDivisor

((-0.0106, -0.06311876657), (0.1478496, 1.020676919), (0.06830916335, 0.4317308477))


In [ ]:
#r "nuget: Accord"
#r "nuget: Accord.Statistics"

open Accord
open Accord.Statistics.Models.Regression.Linear

Microsoft.DotNet.Interactive.InstallPackagesMessage


In [ ]:
type RegData =
    { Date : DateTime
      Portfolio : float
      MktRf : float 
      Hml : float 
      Smb : float }

let ff3ByMonth = 
    ff3
    |> Array.map(fun x -> DateTime(x.Date.Year, x.Date.Month,1), x)
    |> Map

let regDataCreator (returnList: StockData list) =
    let full =
        returnList
          |> List.map (fun x ->
          let xff3 = ff3ByMonth[x.Date]
          { Date = x.Date
            Portfolio = x.Return
            MktRf = xff3.MktRf 
            Hml = xff3.Hml 
            Smb = xff3.Smb })
    full

type RegressionOutput =
    { Model : MultipleLinearRegression 
      TValuesWeights : float array
      TValuesIntercept : float 
      R2: float }

/// Type alias for x, y regression data 
type XY = (float array) array * float array

let fitModel (x: (float array) array, y: float array) =
    let ols = new OrdinaryLeastSquares(UseIntercept=true)
    let estimate = ols.Learn(x,y)
    let mse = estimate.GetStandardError(x,y)
    let se = estimate.GetStandardErrors(mse, ols.GetInformationMatrix())
    let tvaluesWeights = 
        estimate.Weights
        |> Array.mapi(fun i w -> w / se.[i])
    let tvalueIntercept = estimate.Intercept / (se |> Array.last)
    let r2 = estimate.CoefficientOfDetermination(x,y)
    { Model = estimate
      TValuesWeights = tvaluesWeights
      TValuesIntercept = tvalueIntercept  
      R2 = r2 }


In [ ]:
let longRegData1 = regDataCreator long1

let longRegData2 = regDataCreator long2

let longRegData = regDataCreator longListDivisor

let longShortRegData1 = regDataCreator longShort1

let longShortRegData2 = regDataCreator longShort2

let longShortRegData = regDataCreator longShortListDivisor

In [ ]:
let longModelCAPM =
    longRegData
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip 

let longModelCAPM1 =
    longRegData1
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip 

let longModelCAPM2 =
    longRegData2
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip 

let longModelFF3 =
    longRegData
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf; obs.Hml; obs.Smb |], obs.Portfolio)
    |> Array.unzip

let longModelFF31 =
    longRegData1
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf; obs.Hml; obs.Smb |], obs.Portfolio)
    |> Array.unzip

let longModelFF32 =
    longRegData2
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf; obs.Hml; obs.Smb |], obs.Portfolio)
    |> Array.unzip

let longShortModelCAPM =
    longShortRegData
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip 

let longShortModelCAPM1 =
    longShortRegData1
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip 

let longShortModelCAPM2 =
    longShortRegData2
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf|], obs.Portfolio)
    |> Array.unzip 

let longShortModelFF3 =
    longShortRegData
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf; obs.Hml; obs.Smb |], obs.Portfolio)
    |> Array.unzip

let longShortModelFF31 =
    longShortRegData1
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf; obs.Hml; obs.Smb |], obs.Portfolio)
    |> Array.unzip

let longShortModelFF32 =
    longShortRegData2
    |> List.toArray
    |> Array.map(fun obs -> [|obs.MktRf; obs.Hml; obs.Smb |], obs.Portfolio)
    |> Array.unzip

In [ ]:
let longCAPM = longModelCAPM |> fitModel

let longCAPM1 = longModelCAPM1 |> fitModel

let longCAPM2 = longModelCAPM2 |> fitModel

let longFF3 = longModelFF3 |> fitModel

let longFF31 = longModelFF31 |> fitModel

let longFF32 = longModelFF32 |> fitModel

let longShortCAPM = longShortModelCAPM |> fitModel

let longShortCAPM1 = longShortModelCAPM1 |> fitModel

let longShortCAPM2 = longShortModelCAPM2 |> fitModel

let longShortFF3 = longShortModelFF3 |> fitModel

let longShortFF31 = longShortModelFF31 |> fitModel

let longShortFF32 = longShortModelFF32 |> fitModel

In [ ]:
longCAPM

{ Model = y(x0) = 0.9402228381832916*x0 + 0.0006408885771393061
  TValuesWeights = [|73.99787026|]
  TValuesIntercept = 1.09799996
  R2 = 0.9565041555 }


In [ ]:
longCAPM1

{ Model = y(x0) = 0.9120857567018343*x0 + 0.0015058068814036043
  TValuesWeights = [|46.83175227|]
  TValuesIntercept = 1.600939359
  R2 = 0.9464874403 }


In [ ]:
longCAPM2

{ Model = y(x0) = 0.9851742314924327*x0 + -0.0008098394575934682
  TValuesWeights = [|64.21488378|]
  TValuesIntercept = -1.215365326
  R2 = 0.9710353199 }


In [ ]:
longFF3

{ Model = y(x0, x1, x2) = 0.952428530293642*x0 + -0.03479935357648237*x1 + -0.0583181558654963*x2 + 0.000733869892894916
  TValuesWeights = [|72.85944001; -1.956063505; -3.105252187|]
  TValuesIntercept = 1.27965592
  R2 = 0.9584442296 }


In [ ]:
longFF31

{ Model =
   y(x0, x1, x2) = 0.9240592559101148*x0 + -0.022704205558904764*x1 + -0.06471683619556463*x2 + 0.0019302051205179166
  TValuesWeights = [|46.60079083; -0.8462511971; -2.460168372|]
  TValuesIntercept = 2.021520514
  R2 = 0.949016827 }


In [ ]:
longFF32

{ Model =
   y(x0, x1, x2) = 1.009035122041303*x0 + -0.1100868890137592*x1 + -0.041842302267006755*x2 + -0.0016094754992213131
  TValuesWeights = [|65.07658057; -4.872291537; -1.552148134|]
  TValuesIntercept = -2.575830406
  R2 = 0.9764326698 }


In [ ]:
longShortCAPM

{ Model = y(x0) = -0.24036157276314654*x0 + 0.002327241029393522
  TValuesWeights = [|-8.373236305|]
  TValuesIntercept = 1.76482255
  R2 = 0.21970746 }


In [ ]:
longShortCAPM1

{ Model = y(x0) = -0.35510240400579435*x0 + 0.00297400717060843
  TValuesWeights = [|-8.75688291|]
  TValuesIntercept = 1.518583972
  R2 = 0.3821100888 }


In [ ]:
longShortCAPM2

{ Model = y(x0) = -0.07915441769796425*x0 + -0.0004130655932166594
  TValuesWeights = [|-2.111884325|]
  TValuesIntercept = -0.2537460215
  R2 = 0.03499178929 }


In [ ]:
longShortFF3

{ Model =
   y(x0, x1, x2) = -0.20215703222614714*x0 + 0.011745451792397067*x1 + -0.18313093260770905*x2 + 0.0025215653078637873
  TValuesWeights = [|-6.965226721; 0.2973544704; -4.391852771|]
  TValuesIntercept = 1.980328927
  R2 = 0.2799960904 }


In [ ]:
longShortFF31

{ Model =
   y(x0, x1, x2) = -0.3153602661871545*x0 + 0.07323458967165067*x1 + -0.17125887163840683*x2 + 0.0032600758928834016
  TValuesWeights = [|-7.983305984; 1.370224015; -3.268006922|]
  TValuesIntercept = 1.71389707
  R2 = 0.4611102042 }


In [ ]:
longShortFF32

{ Model =
   y(x0, x1, x2) = -0.013026417883506612*x0 + -0.2068516825275521*x1 + -0.1684965098464489*x2 + -0.0021407535295370123
  TValuesWeights = [|-0.3366727542; -3.668776449; -2.504799847|]
  TValuesIntercept = -1.372979612
  R2 = 0.1808032267 }


In [ ]:
type Prediction = { Label : float; Score : float}

let makePredictions 
    (estimate:MultipleLinearRegression) 
    (x: (float array) array, y: float array) =
    (estimate.Transform(x), y)
    ||> Array.zip
    |> Array.map(fun (score, label) -> { Score = score; Label = label })

let residuals (xs: Prediction array) = xs |> Array.map(fun x -> x.Label - x.Score)

let informationRatio monthlyAlpha (monthlyResiduals: float array) =
    let annualAlpha = 12.0 * monthlyAlpha
    let annualStDev = sqrt(12.0) * (Seq.stDev monthlyResiduals)
    annualAlpha / annualStDev

In [ ]:
let predictionsLongCAPM = makePredictions longCAPM.Model longModelCAPM

let predictionsLongCAPM1 = makePredictions longCAPM1.Model longModelCAPM1

let predictionsLongCAPM2 = makePredictions longCAPM2.Model longModelCAPM2

let predictionsLongFF3 = makePredictions longFF3.Model longModelFF3

let predictionsLongFF31 = makePredictions longFF31.Model longModelFF31

let predictionsLongFF32 = makePredictions longFF32.Model longModelFF32

let predictionsLongShortCAPM = makePredictions longShortCAPM.Model longShortModelCAPM

let predictionsLongShortCAPM1 = makePredictions longShortCAPM1.Model longShortModelCAPM1

let predictionsLongShortCAPM2 = makePredictions longShortCAPM2.Model longShortModelCAPM2

let predictionsLongShortFF3 = makePredictions longShortFF3.Model longShortModelFF3

let predictionsLongShortFF31 = makePredictions longShortFF31.Model longShortModelFF31

let predictionsLongShortFF32 = makePredictions longShortFF32.Model longShortModelFF32

In [ ]:
let residualsLongCAPM = residuals predictionsLongCAPM

let residualsLongCAPM1 = residuals predictionsLongCAPM1

let residualsLongCAPM2 = residuals predictionsLongCAPM2

let residualsLongFF3 = residuals predictionsLongFF3

let residualsLongFF31 = residuals predictionsLongFF31

let residualsLongFF32 = residuals predictionsLongFF32

let residualsLongShortCAPM = residuals predictionsLongShortCAPM

let residualsLongShortCAPM1 = residuals predictionsLongShortCAPM1

let residualsLongShortCAPM2 = residuals predictionsLongShortCAPM2

let residualsLongShortFF3 = residuals predictionsLongFF3

let residualsLongShortFF31 = residuals predictionsLongShortFF31

let residualsLongShortFF32 = residuals predictionsLongShortFF32

In [ ]:
informationRatio longCAPM.Model.Intercept residualsLongCAPM, informationRatio longCAPM1.Model.Intercept residualsLongCAPM1, informationRatio longCAPM2.Model.Intercept residualsLongCAPM2

(0.2424301568, 0.4961317872, -0.3942924129)


In [ ]:
informationRatio longFF3.Model.Intercept residualsLongFF3, informationRatio longFF31.Model.Intercept residualsLongFF31, informationRatio longFF32.Model.Intercept residualsLongFF32

(0.2840085388, 0.6515468824, -0.8687264254)


In [ ]:
informationRatio longShortCAPM.Model.Intercept residualsLongShortCAPM, informationRatio longShortCAPM1.Model.Intercept residualsLongShortCAPM1, informationRatio longShortCAPM2.Model.Intercept residualsLongShortCAPM2

(0.3896595842, 0.470609818, -0.08232103462)


In [ ]:
informationRatio longShortFF3.Model.Intercept residualsLongShortFF3, informationRatio longShortFF31.Model.Intercept residualsLongShortFF31, informationRatio longShortFF32.Model.Intercept residualsLongShortFF32

(0.9758488329, 0.5523981995, -0.4630520968)


## Strategy as part of a diversified portfolio

### Tangency Portfolios

In [ ]:
let tickers = 
    [   "VTI" // Vanguard Total Stock Market ETF
        "BND" // Vanguard Total Bond Market ETF
    ]

let tickPrices = 
    YahooFinance.PriceHistory(
        tickers,
        startDate = DateTime(2000,1,1),
        interval = Monthly)

let pricesToReturns (symbol, adjPrices: list<PriceObs>) =
    adjPrices
    |> List.sortBy (fun x -> x.Date)
    |> List.pairwise
    |> List.map (fun (day0, day1) ->
        let r = day1.AdjustedClose / day0.AdjustedClose - 1.0 
        { Symbol = symbol 
          Date = day1.Date 
          Return = r })

let tickReturns =
    tickPrices
    |> List.groupBy (fun x -> x.Symbol)
    |> List.collect pricesToReturns

In [ ]:
let rf = Map [ for x in ff3 do x.Date, x.Rf ]

let standardInvestmentsExcess =
    let maxff3Date = ff3 |> Array.map(fun x -> x.Date) |> Array.max
    tickReturns
    |> List.filter(fun x -> x.Date <= maxff3Date)
    |> List.map(fun x -> 
        match Map.tryFind x.Date rf with 
        | None -> failwith $"why isn't there a rf for {x.Date}"
        | Some rf -> { x with Return = x.Return - rf }) // excess returns

In [ ]:
let stockData =
    standardInvestmentsExcess
    |> List.groupBy(fun x -> x.Symbol)
    |> Map

In [ ]:
let getCov xId yId (optimizedData: Map<string,StockData list>) =
    let xRet = 
        optimizedData[xId] 
        |> List.map (fun x -> x.Date,x.Return) 
        |> Map
    let yRet = 
        optimizedData[yId]
        |> List.map (fun y -> y.Date, y.Return)
        |> Map
    let overlappingDates =
        [ xRet.Keys |> set
          yRet.Keys |> set]
        |> Set.intersectMany
    [ for date in overlappingDates do xRet[date], yRet[date]]
    |> Seq.covOfPairs

In [ ]:
let longOptimal = List.append longListDivisor standardInvestmentsExcess

let longOptimalGrouped =
    longOptimal
    |> List.groupBy(fun x -> x.Symbol)
    |> Map

let longShortOptimal = List.append longShortListDivisor standardInvestmentsExcess

let longShortOptimalGrouped = 
    longShortOptimal
    |> List.groupBy(fun x -> x.Symbol)
    |> Map

In [ ]:
let longTickers = 
    [ 
        "Long" 
        "VTI" 
        "BND" 
    ]

let longShortTickers = 
    [ 
        "Long Short" 
        "VTI" 
        "BND" 
    ]

In [ ]:
let longCovariances =
    [ for rowTick in longTickers do 
        [ for colTick in longTickers do
            getCov rowTick colTick longOptimalGrouped ]]
    |> dsharp.tensor

let longShortCovariances =
    [ for rowTick in longShortTickers do 
        [ for colTick in longShortTickers do
            getCov rowTick colTick longShortOptimalGrouped ]]
    |> dsharp.tensor

let longMeans =
    [ for ticker in longTickers do 
        longOptimalGrouped[ticker]
        |> List.averageBy (fun x -> x.Return)]
    |> dsharp.tensor

let longShortMeans =
    [ for ticker in longShortTickers do 
        longShortOptimalGrouped[ticker]
        |> List.averageBy (fun x -> x.Return)]
    |> dsharp.tensor

In [ ]:
let w' = dsharp.solve(longCovariances,longMeans)

let wLong = w' / w'.sum()

In [ ]:
let w' = dsharp.solve(longShortCovariances,longShortMeans)

let wLongShort = w' / w'.sum()

In [ ]:
let longPortVariance = wLong.matmul(longCovariances).matmul(wLong)

let longPortStDev = longPortVariance.sqrt()

let longPortMean = dsharp.matmul(wLong, longMeans)

let longShortPortVariance = wLong.matmul(longShortCovariances).matmul(wLongShort)

let longShortPortStDev = longShortPortVariance.sqrt()

let longShortPortMean = dsharp.matmul(wLongShort, longShortMeans)

In [ ]:
let longWeights =
    Seq.zip longTickers (wLong.toArray1D<float>())
    |> Map.ofSeq

let longShortWeights =
    Seq.zip longShortTickers (wLongShort.toArray1D<float>())
    |> Map.ofSeq

In [ ]:
let longOptimalGroupedDate =
    longOptimalGrouped.Values
    |> Seq.toList
    |> List.collect id 
    |> List.groupBy(fun x -> x.Date) 
    |> List.sortBy fst 

let longShortOptimalGroupedDate =
    longShortOptimalGrouped.Values
    |> Seq.toList
    |> List.collect id 
    |> List.groupBy(fun x -> x.Date) 
    |> List.sortBy fst 

In [ ]:
let longFirstMonth = 
    longOptimalGroupedDate 
    |> List.head 
    |> snd // convert (date, StockData list) -> StockData list

let longShortFirstMonth = 
    longShortOptimalGroupedDate 
    |> List.head 
    |> snd // convert (date, StockData list) -> StockData list

let longLastMonth = 
    longOptimalGroupedDate 
    |> List.last 
    |> snd // convert (date, StockData list) -> StockData list

let longShortLastMonth = 
    longShortOptimalGroupedDate 
    |> List.last 
    |> snd // convert (date, StockData list) -> StockData list

In [ ]:
let allAssetsStart =
    longOptimalGroupedDate
    // find the first array element where there are as many stocks as you have symbols
    |> List.find(fun (month, stocks) -> stocks.Length = longTickers.Length)
    |> fst // convert (month, stocks) to month

let allAssetsEnd =
    longOptimalGroupedDate
    // find the last array element where there are as many stocks as you have symbols
    |> List.findBack(fun (month, stocks) -> stocks.Length = longTickers.Length)
    |> fst // convert (month, stocks) to month

let longStockDataByDateComplete =
    longOptimalGroupedDate
    |> List.filter(fun (date, stocks) -> 
        date >= allAssetsStart &&
        date <= allAssetsEnd)

let longShortStockDataByDateComplete =
    longShortOptimalGroupedDate
    |> List.filter(fun (date, stocks) -> 
        date >= allAssetsStart &&
        date <= allAssetsEnd)

let longCheckOfCompleteData =
    longStockDataByDateComplete
    |> List.map snd
    |> List.filter(fun x -> x.Length <> longTickers.Length) // discard rows where we have all symbols.

let longShortCheckOfCompleteData =
    longShortStockDataByDateComplete
    |> List.map snd
    |> List.filter(fun x -> x.Length <> longShortTickers.Length) // discard rows where we have all symbols.

In [ ]:
if not (List.isEmpty longCheckOfCompleteData) then 
        failwith "stockDataByDateComplete has months with missing stocks"

In [ ]:
if not (List.isEmpty longShortCheckOfCompleteData) then 
        failwith "stockDataByDateComplete has months with missing stocks"

### Diversified portfolio comparison

In [ ]:
let portfolioMonthReturn weights monthData =
    weights
    |> Map.toList
    |> List.map(fun (symbol, weight) ->
        let symbolData = 
            // we're going to be more safe and use tryFind here so
            // that our function is more reusable
            match monthData |> List.tryFind(fun x -> x.Symbol = symbol) with
            | None -> failwith $"You tried to find {symbol} in the data but it was not there"
            | Some data -> data
        symbolData.Return*weight)
    |> List.sum    

In [ ]:
let longPortMve = 
    longStockDataByDateComplete
    |> List.map(fun (date, data) -> 
        { Symbol = "MVELong"
          Date = date
          Return = portfolioMonthReturn longWeights data })

let longShortPortMve = 
    longShortStockDataByDateComplete
    |> List.map(fun (date, data) -> 
        { Symbol = "MVELong"
          Date = date
          Return = portfolioMonthReturn longShortWeights data })

let weights6040 = Map [("VTI",0.6);("BND",0.4)]

let port6040 = 
    longStockDataByDateComplete
    |> List.map(fun (date, data) -> 
        { Symbol = "MVELong"
          Date = date
          Return = portfolioMonthReturn weights6040 data })


In [ ]:
let cumulateReturns (xs:list<StockData>) =
    let folder (prev: StockData) (current: StockData) = 
        let newReturn = prev.Return * (1.0+current.Return)
        { current with Return = newReturn}
    
    match xs |> List.sortBy (fun x -> x.Date) with
    | [] -> []
    | h::t ->
        ({ h with Return = 1.0+ h.Return}, t) 
        ||> List.scan folder

let longPortMveCumulative = 
    longPortMve
    |> cumulateReturns

let longShortPortMveCumulative = 
    longShortPortMve
    |> cumulateReturns

let port6040Cumulative = 
    port6040
    |> cumulateReturns

In [ ]:
let longChartMve = 
    longPortMveCumulative
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVELong")

let longShortChartMve = 
    longShortPortMveCumulative
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVELongShort")

let chart6040 = 
    port6040Cumulative
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="60/40")

let chartStrategyAnalysisDiversified =
    [ longChartMve; longShortChartMve; chart6040 ]
    |> Chart.combine
    |> Chart.withTitle "Strategy Analysis Diversified: Market, Short and Long-Short"

chartStrategyAnalysisDiversified

<!-- Plotly chart will be drawn inside this DIV -->

In [ ]:
let normalize10pctVol xs =
    let vol = xs |> List.map(fun x -> x.Return) |> Seq.stDev
    let annualizedVol = vol * sqrt(12.0)
    xs 
    |> List.map(fun x -> { x with Return = x.Return * (0.1/annualizedVol)})

let longPortMveCumulative10 = 
    longPortMve
    |> normalize10pctVol
    |> cumulateReturns

let longShortPortMveCumulative10 = 
    longShortPortMve
    |> normalize10pctVol
    |> cumulateReturns

let port6040Cumulative10 = 
    port6040
    |> normalize10pctVol
    |> cumulateReturns

In [ ]:
let longChartMve10 = 
    longPortMveCumulative10
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVELong")

let longShortChartMve10 = 
    longShortPortMveCumulative10
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="MVELongShort")

let chart604010 = 
    port6040Cumulative10
    |> List.map(fun x -> x.Date, x.Return)
    |> Chart.Line
    |> Chart.withTraceInfo(Name="60/40")

let chartStrategyAnalysisDiversified10 =
    [ longChartMve10; longShortChartMve10; chart604010 ]
    |> Chart.combine
    |> Chart.withTitle "Strategy Analysis Diversified:(10% volatility)"

chartStrategyAnalysisDiversified10

<!-- Plotly chart will be drawn inside this DIV -->

### Performance analysis

In [ ]:
let longReturnMve =
    longPortMve
    |> Seq.averageBy (fun x -> x.Return)

let longShortReturnMve =
    longShortPortMve
    |> Seq.averageBy (fun x -> x.Return)

let return6040 =
    port6040
    |> Seq.averageBy (fun x -> x.Return)

let longAnnualizedReturnMve = averageAnnualizedExcessReturn longReturnMve

let longShortAnnualizedReturnMve = averageAnnualizedExcessReturn longShortReturnMve

let annualizedReturn6040 = averageAnnualizedExcessReturn return6040

In [ ]:
longAnnualizedReturnMve, longShortAnnualizedReturnMve, annualizedReturn6040

(0.04699881034, 0.03587153535, 0.07215323757)


In [ ]:
let longStDevMve = 
    longPortMve
    |> Seq.stDevBy (fun x -> x.Return)

let longShortStDevMve = 
    longPortMve
    |> Seq.stDevBy (fun x -> x.Return)

let stDev6040 = 
    port6040
    |> Seq.stDevBy (fun x -> x.Return)

let longSharpeMve = longReturnMve / longStDevMve

let longShortSharpeMve  = longShortReturnMve / longShortStDevMve

let sharpe6040  = return6040 / stDev6040

let longAnnualizedSharpeRatioMve = annualizedSharpeRatio longSharpeMve

let longShortAnnualizedSharpeRatioMve = annualizedSharpeRatio longShortSharpeMve

let Annualized6040SharpeRatioMve = annualizedSharpeRatio sharpe6040

In [ ]:
longAnnualizedSharpeRatioMve, longShortAnnualizedSharpeRatioMve, Annualized6040SharpeRatioMve

(1.087501115, 0.8300281303, 0.7109978563)
